# Make parameter sweep submission script for use on HPC using Multiple Serial ($\delta$-sweep)

For simplicity, $\kappa = 0$ for all of these parametrizations

Author: Juvid Aryaman

Date: 06/09/17

In [1]:
import numpy as np
import pandas as pd
import sing_birth_fus_utils_ap as utls
import pdb

%reload_ext autoreload
%autoreload 2

In [2]:
work_dir = '/work/ja1109/networks/sing_birth_fus/param_sweep/delta_sweep' # directory on HPC where the work will be done

In [3]:
# Param ordering convention
# xi, beta, gamma, kappa, b, mu, delta, h_target
param_sw_vals = pd.read_csv('../Make_sweep_vals/delta_sw_params.csv')
param_sw_vals[['ws_init','wf_init','ms_init','mf_init']] = param_sw_vals[['ws_init','wf_init','ms_init','mf_init']].astype(int)

param_sw_vals



,xi,beta,gamma,kappa,b,mu,delta,h_target,ws_init,wf_init,ms_init,mf_init
0,0.0,33.12,0.037851,0.0,0.000012,0.023,-5.00,0.10,639,2749,71,306
1,0.0,33.12,0.037851,0.0,0.000012,0.023,-1.00,0.30,497,2137,213,917
2,0.0,33.12,0.037851,0.0,0.000012,0.023,-0.10,0.45,367,1169,299,954
3,0.0,33.12,0.037851,0.0,0.000012,0.023,-0.01,0.45,357,1029,292,843
4,0.0,33.12,0.037851,0.0,0.000012,0.023,0.01,0.45,355,1001,290,818
5,0.0,33.12,0.037851,0.0,0.000012,0.023,0.10,0.45,346,886,283,725
6,0.0,33.12,0.037851,0.0,0.000012,0.023,1.00,0.30,323,362,138,155
7,0.0,33.12,0.037851,0.0,0.000012,0.023,1.50,0.45,203,148,166,121
8,0.0,33.12,0.037851,0.0,0.000012,0.023,2.00,0.20,303,232,76,58


In [4]:
# parameter ordering convention in the .c script
param_order = ['ws_init','ms_init','wf_init','mf_init','xi','beta',	'gamma','kappa','b','mu','delta']

In [5]:
n_iters_per_block = 1000 # multiply by NREPEATS in the .c script for the total number of iterations
#n_iters_per_block = 5

In [6]:
initial_seed = 576967 #np.round(np.random.uniform()*1e6).astype(int)
print(initial_seed)

576967


In [8]:
time_per_job = '1:00:00' # slowest iter took 30s. 30*10*10 = 3000 s = 50 mins

Submission protocol

On local machine

`$python Make_submission_param_sweep.py` 

(do this locally because some work must be done to find the right settings)

`$rsyncr * hpc:$PBS_O_WORKDIR`


where `$PBS_O_WORKDIR` is the directory where you run the script from.

`$gcc -o3 closed_loop_deg_single.c -lm -o closed_loop_deg_single.ce`

`$bash master.sh`

In [9]:
def make_submission_string(initial_seed, param_block, n_iters_per_block, params):
    submission_string = './closed_loop_deg_single.ce {0} {1} {2} $PBS_ARRAY_INDEX'.format(initial_seed,
                                                                                             param_block,
                                                                                             n_iters_per_block)     
    for i, p in enumerate(params):
        if i <= 3:
            p = int(round(p))
        submission_string+=' '+str(p)

    submission_string+='\n'

    return submission_string

In [10]:
def make_submission_script(param_block,time_per_job,n_iters_per_block,initial_seed, params):
    f = open('submission_{}.pbs'.format(param_block),'w')
    f.write('#!/bin/sh\n')
    f.write('#PBS -N CLC_SW_{}\n'.format(param_block))
    f.write('#PBS -l walltime={}\n'.format(time_per_job))
    f.write('#PBS -l select=1:ncpus=1:mem=1gb\n')
    f.write('#PBS -J 0-{}\n'.format(n_iters_per_block - 1))
    
    f.write('cp $PBS_O_WORKDIR/closed_loop_deg_single.ce $TMPDIR\n')
    
    submission_string = make_submission_string(initial_seed, param_block, n_iters_per_block, params)
      
    f.write(submission_string) 
    f.write('cp output_{0}_$PBS_ARRAY_INDEX.txt $PBS_O_WORKDIR/p{0:03d}\n'.format(param_block))
    
    # clean up -- have a file number quota
    f.write('sleep 10\n')
    f.write('rm $PBS_O_WORKDIR/CLC_SW_*\n')
    f.close()

In [11]:
master_script = open('master.sh','w')
for row in param_sw_vals.iterrows():
    idx = row[0]
    vals = row[1]
        
    params_subm = vals[param_order].as_matrix()
    
    
    make_submission_script(idx,time_per_job,n_iters_per_block,initial_seed,params_subm)
    master_script.write('echo "{}"\n'.format(idx))
    master_script.write('mkdir -p {0}/p{1:03d}\n'.format(work_dir,idx))
    master_script.write("qsub submission_{}.pbs\n".format(idx))
    master_script.write('sleep 0.5\n')
    print(idx)
            
master_script.close()


0
1
2
3
4
5
6
7
8
